# <center> ***ML-6 Отбор признаков (HW-03)***

In [1]:
# Импортируем используемые библиотеки.
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
# Загружаем данные.
data = pd.read_excel('data/data_ford_price.xlsx') 
data

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7012,22500,2015,3,6,23500,clean,automatic,rwd,full-size,32.680700,-117.169800,59.0
7013,5975,2005,2,8,0,clean,automatic,rwd,full-size,38.213303,-85.785762,50.0
7014,9999,2006,3,8,161514,clean,automatic,NaN,full-size,37.609783,-120.995406,59.0
7015,10900,2011,2,8,164000,clean,automatic,4wd,full-size,43.140600,-93.385000,47.0


In [3]:
# Удаляем пропуски в данных.
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

# Отделяем из датасет целевой признак.
y = data['price']
x = data.drop(columns='price')

# Разбиваем данные на тренировочную и тестовую.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

/var/folders/46/1dnpmgs97ll3y5gzzrlpx3d80000gn/T/ipykernel_8150/3615282415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace = True)


## ***Метод рекурсивного исключения признаков.***

In [4]:
# Находим 3 наиболее значимых признака.
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
rfe_feature = selector.get_feature_names_out().tolist()
rfe_feature

['year', 'cylinders', 'lat']

## ***Метод выбора признака на основе фильтров.***

In [5]:
# Находим 3 наиболее значимых признака.
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
filter_feature = selector.get_feature_names_out().tolist()
filter_feature

['year', 'cylinders', 'odometer']

## ***Обучение моделей.***

### 1. Обучение методом рекурсивного исключения признаков.

In [6]:
# Выделим в отдельный датасет значимые признаки.
x_rfe = data[rfe_feature]

# Разбиваем данные на тренировочную и тестовую.
X_train, X_test, y_train, y_test = train_test_split(x_rfe, y, test_size=0.3, random_state=40)

# Обучаем модель.
rfe_model = LinearRegression()
rfe_model.fit(X_train, y_train)
y_predicted_rfe = rfe_model.predict(X_test)
 
print('MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_predicted_rfe)))
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, y_predicted_rfe)))

MAE: 5096.570
R2 score: 0.576


### 2. Обучение методом выбора признака на основе фильтров.

In [7]:
# Выделим в отдельный датасет значимые признаки.
x_filter = data[filter_feature]

# Разбиваем данные на тренировочную и тестовую.
X_train, X_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.3, random_state=40)

# Обучаем модель.
filter_model = LinearRegression()
filter_model.fit(X_train, y_train)
y_predicted_filter = filter_model.predict(X_test)
 
print('MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_predicted_filter)))
print('R2 score: {:.3f}'.format(metrics.r2_score(y_test, y_predicted_filter)))

MAE: 4708.946
R2 score: 0.602


## ***Вывод:***


*Проведя отбор признаков методом рекурсивного исключения и методом выбора на основе фильтров мы получили следующие метрики после обучения моделей:*
| Metrics/ metod | RFE      | SelectKBest |
|----------------|----------|-------------|
| MAE            | 5096.570 | 4708.946    |
| R2             | 0.576    | 0.602       | 

*Как мы выдим из данных, с помощью метода выбора признаков на основе фильтров выбранные метрики показывают лучше результат, а именно средняя абсолютная ошибка снизилась почти на 390$ и в среднем данная модель расходится с реальными данными на 4709$, а коэффициент детерминации увеличился с 0.576 до 0.602.*

*Таким образом с предоставленными данными в нашем датасете метод выбота признаков на основе фильтров справился лучше для нахождения наиболее значимых признаков.*